In [1]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
spark = SparkSession.builder.appName('fun').getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

# Importing vertices and edges from csv files

In [2]:
vertices = spark.read.option("header",True).csv("vertices.csv").distinct()
edges = spark.read.option("header",True).csv("edges.csv").distinct()
g = GraphFrame(vertices, edges)

# Finding all paths between 2 nodes

Companies in a direct contact relationship

In [4]:
%%time
paths = g.bfs(fromExpr="id = 'SCMM'", toExpr="id = 'TRACO COMPUTERS S.R.O.'", maxPathLength=10).show(100, truncate=100, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------
 from | {SCMM}                                                                                               
 e0   | {SCMM, TRACO COMPUTERS S.R.O., contract, https://www.crz.gov.sk/116549/, 116549, KaVSÚ-1-9/2011, ... 
 to   | {TRACO COMPUTERS S.R.O.}                                                                             

CPU times: user 19.4 ms, sys: 10.3 ms, total: 29.7 ms
Wall time: 14.7 s


Companies on the same address

In [6]:
%%time
paths = g.bfs(fromExpr="id = 'SLOVAK TELEKOM, A. S.'", toExpr="id = 'T MOBILE SLOVAK TELEKOM, A.S.'", maxPathLength=5).show(1, truncate=100, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------
 from | {SLOVAK TELEKOM, A. S.}                                                                              
 e0   | {SLOVAK TELEKOM, A. S., Karadžičova 10, street, https://www.crz.gov.sk/156241/, 156241, 8/2011, M... 
 v1   | {Karadžičova 10}                                                                                     
 e1   | {Karadžičova 10, T MOBILE SLOVAK TELEKOM, A.S., street, https://www.crz.gov.sk/141478/, 141478, 2... 
 to   | {T MOBILE SLOVAK TELEKOM, A.S.}                                                                      
only showing top 1 row

CPU times: user 37.4 ms, sys: 27.4 ms, total: 64.8 ms
Wall time: 27.9 s


Companies in the same city

In [8]:
%%time
paths = g.bfs(fromExpr="id = 'V. D. S. SPOL. S R. O.'", toExpr="id = 'ENBRA SLOVAKIA S.R.O.'", maxPathLength=5).show(1, truncate=100, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------
 from | {V. D. S. SPOL. S R. O.}                                                                             
 e0   | {V. D. S. SPOL. S R. O., Banská Bystrica, city, https://www.crz.gov.sk/124838/, 124838, KRPZ-BB_D... 
 v1   | {Banská Bystrica}                                                                                    
 e1   | {Banská Bystrica, ENBRA SLOVAKIA S.R.O., city, https://www.crz.gov.sk/124872/, 124872, ZoD 02/201... 
 to   | {ENBRA SLOVAKIA S.R.O.}                                                                              
only showing top 1 row

CPU times: user 41.9 ms, sys: 22.6 ms, total: 64.6 ms
Wall time: 25.8 s


Company closed a contract with another company on the same address

In [10]:
%%time
paths = g.bfs(fromExpr="id = 'SLOVENSKÁ POĽNOHOSPODÁRSKA UNIVERZITA V NITRE'", toExpr="id = 'T MOBILE SLOVAK TELEKOM, A.S.'", maxPathLength=5).show(1, truncate=100, vertical=True)

-RECORD 0----------------------------------------------------------------------------------------------------
 from | {SLOVENSKÁ POĽNOHOSPODÁRSKA UNIVERZITA V NITRE}                                                      
 e0   | {SLOVENSKÁ POĽNOHOSPODÁRSKA UNIVERZITA V NITRE, SLOVAK TELEKOM, A.S., contract, https://www.crz.g... 
 v1   | {SLOVAK TELEKOM, A.S.}                                                                               
 e1   | {SLOVAK TELEKOM, A.S., Karadžičova 10, street, https://www.crz.gov.sk/143717/, 143717, SITB_D04_S... 
 v2   | {Karadžičova 10}                                                                                     
 e2   | {Karadžičova 10, T MOBILE SLOVAK TELEKOM, A.S., street, https://www.crz.gov.sk/141478/, 141478, 2... 
 to   | {T MOBILE SLOVAK TELEKOM, A.S.}                                                                      
only showing top 1 row

CPU times: user 94.7 ms, sys: 40.1 ms, total: 135 ms
Wall time: 1min 24s


In [20]:
# Search for pairs of vertices with edges in both directions between them.
#motifs = g.find("(a)-[e]->(b); (b)-[e2]->(c)").filter("e.type = 'contract'").filter("e2.type = 'street'").filter("a.id = 'SLOVENSKÁ POĽNOHOSPODÁRSKA UNIVERZITA V NITRE'").filter("b.id = 'T MOBILE SLOVAK TELEKOM, A.S.'").select("e.src", "e.dst", "e.type").show()

+---+---+----+
|src|dst|type|
+---+---+----+
+---+---+----+

